# Classification Results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.anova import AnovaRM
import statsmodels.api as sm 
import pingouin as pg
import scikit_posthocs as sp
import os

In [ ]:
#ChatGPT provided method for extracting data
from tbparse import SummaryReader

relative_path = os.path.curdir

reader = SummaryReader(relative_path + "/cnnweak_['drsbru']_20250515-180608")
df_drsbru_acc = reader.scalars

reader = SummaryReader(relative_path + "/cnnweak_['drsprg', 'drsbru']_20250515-181939")
df_both_acc = reader.scalars
df_both_pr = reader.text

reader = SummaryReader(relative_path + "/cnnweak_['drsprg']_20250515-174725")
df_drsprg_acc = reader.scalars

In [ ]:
df_both_acc.head()

In [ ]:
#Process data

df_drsprg_acc['tag'] = df_drsprg_acc['tag'].str.replace('acc', '')
df_drsprg_acc['radiomics'] = df_drsprg_acc['tag'].str.contains(r'RADIOMICS|rad').map({True: 'True', False: 'False'})
df_drsprg_acc['model'] = df_drsprg_acc['tag'].str.replace(r'RADIOMICS|rad', '', regex=True)
df_drsprg_acc['model'] = df_drsprg_acc['model'].str.replace("3D CNN +", "")
df_drsprg_acc['model'] = df_drsprg_acc['model'].str.replace("BASELINE", "2D ResNet18")
df_drsprg_acc['model'] = df_drsprg_acc['model'].str.strip()

#Total highest accuracy
model_order = ["2D ResNet18", "NN", "KNN", "SVM", "ET", "RF", "Ensemble"]

In [ ]:
fig = plt.figure(figsize=(8, 5))

# Create the barplot with SD error bars
sns.barplot(
    data=df_drsprg_acc,
    x='model',
    y='value',
    hue='radiomics',
    palette='Purples',
    estimator=np.mean,
    err_kws={"color": "0", "linewidth": 1.5},
    width=0.7,
    errorbar="sd", capsize=.1,
    linewidth=1.5, edgecolor="0",
    order=model_order
)


#plt.xticks(rotation=45)
plt.ylabel("Accuracy")
plt.xlabel("Classification Models")
plt.title("10-Fold CV Accuracies With drsprg Data")
plt.legend(title="Radiomic Features")
plt.grid()
plt.ylim(0, 0.9)
plt.tight_layout()
plt.savefig('drsprg_accuracy.pdf', bbox_inches='tight')
plt.show()

# DRSBRU

In [ ]:

df_drsbru_acc['tag'] = df_drsbru_acc['tag'].str.replace('acc', '')
df_drsbru_acc['radiomics'] = df_drsbru_acc['tag'].str.contains(r'RADIOMICS|rad').map({True: 'True', False: 'False'})
df_drsbru_acc['model'] = df_drsbru_acc['tag'].str.replace(r'RADIOMICS|rad', '', regex=True)
df_drsbru_acc['model'] = df_drsbru_acc['model'].str.replace("3D CNN +", "")
df_drsbru_acc['model'] = df_drsbru_acc['model'].str.replace("BASELINE", "2D ResNet18")
df_drsbru_acc['model'] = df_drsbru_acc['model'].str.strip()

In [ ]:
fig = plt.figure(figsize=(8, 5))
# Create the barplot with SD error bars
sns.barplot(
    data=df_drsbru_acc,
    x='model',
    y='value',
    hue='radiomics',
    palette='Greens',
    estimator=np.mean,
    err_kws={"color": "0", "linewidth": 1.5},
    width=0.7,
    errorbar="sd", capsize=.1,
    linewidth=1.5, edgecolor="0",
    order=model_order
)


#plt.xticks(rotation=45)
plt.ylabel("Accuracy")
plt.xlabel("Classification Models")
plt.title("10-Fold CV Accuracies With drsbru Data")
plt.legend(title="Radiomic Features")
plt.grid()
plt.ylim(0, 0.9)
plt.tight_layout()
plt.savefig('drsbru_accuracy.pdf', bbox_inches='tight')
plt.show()

# Both

In [ ]:
df_both_acc['tag'] = df_both_acc['tag'].str.replace('acc', '')
df_both_acc['radiomics'] = df_both_acc['tag'].str.contains(r'RADIOMICS|rad').map({True: 'True', False: 'False'})
df_both_acc['model'] = df_both_acc['tag'].str.replace(r'RADIOMICS|rad', '', regex=True)
df_both_acc['model'] = df_both_acc['model'].str.replace("3D CNN +", "")
df_both_acc['model'] = df_both_acc['model'].str.replace("BASELINE", "2D ResNet18").str.strip()
df_both_acc['model'] = df_both_acc['model'].str.strip()
df_both_acc

In [ ]:
fig = plt.figure(figsize=(8, 5))

# Create the barplot with SD error bars
sns.barplot(
    data=df_both_acc,
    x='model',
    y='value',
    hue='radiomics',
    palette='Blues',
    estimator=np.mean,
    err_kws={"color": "0", "linewidth": 1.5},
    width=0.7,
    errorbar="sd", capsize=.1,
    linewidth=1.5, edgecolor="0",
    order=model_order
)


#plt.xticks(rotation=45)
plt.ylabel("Accuracy")
plt.xlabel("Classification Models")
plt.title("10-Fold CV Accuracies With drsbru and drsprg Data")
plt.legend(title="Radiomic Features")
plt.grid()
plt.ylim(0, 0.9)
plt.tight_layout()
plt.savefig('both_accuracy.pdf', bbox_inches='tight')
plt.show()

Violin Plot with both datasets

In [ ]:
fig = plt.figure(figsize=(8, 5))
sns.violinplot(
    data=df_both_acc,
    palette='Blues',
    x='model',
    y='value',
    hue='radiomics',
    split=True,
    order=model_order
)
plt.title("10-Fold CV Accuracies With drsbru and drsprg Data")
plt.ylabel("Accuracy")
plt.xlabel("Model")
plt.grid()
plt.savefig('violin_combined.pdf', bbox_inches='tight')
plt.show()

# Checking for normally distributed data

In [ ]:
df_both_stat = df_both_acc[df_both_acc["radiomics"] == "False"].pivot(index="step", columns="model", values="value")
df_both_stat

In [ ]:
for model in df_both_stat.loc[:, df_both_stat.columns != "Step"]:
    result = stats.shapiro(df_both_stat[model])
    print(model)
    print(result)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 10))
axes = axes.flatten()
for i in range(7):
        model = df_both_stat.columns[i]
        stats.probplot(df_both_stat[model], dist="norm", plot=axes[i])
        axes[i].set_title(f"Q-Q Plot: {model}")

# Hide unused axes (last two)
for j in range(7, 9):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.savefig('QQ_plot_class.pdf', bbox_inches='tight')

## Friedman

In [ ]:
res = stats.friedmanchisquare(df_both_stat["2D ResNet18"], df_both_stat["ET"], df_both_stat["Ensemble"], df_both_stat["KNN"], df_both_stat["NN"], df_both_stat["RF"], df_both_stat["SVM"])
print(res)

### Nemenyi test

In [ ]:
sp.posthoc_nemenyi_friedman(df_both_stat)

## RM ANOVA

In [ ]:
df_both_long = df_both_acc
rm_anova = pg.rm_anova(df_both_long, dv='value', subject='step', within='model', correction=True).round(3)
rm_anova

In [ ]:
bonferroni = pg.pairwise_tests(dv='value', within='model', subject='step', padjust='bonf', data=df_both_long)
bonferroni

# MEAN VALUES - DRSPRG

In [ ]:
mean_std_drsprg = df_drsprg_acc.groupby(['radiomics', 'model'])['value'].agg(['mean', 'std']).reset_index()
mean_std_drsprg['mean'] = mean_std_drsprg['mean'].round(2)
mean_std_drsprg['std'] = mean_std_drsprg['std'].round(2)
mean_std_drsprg

# MEAN VALUES - DRSBRU

In [ ]:
mean_std_drsbru = df_drsbru_acc.groupby(['radiomics', 'model'])['value'].agg(['mean', 'std']).reset_index()
mean_std_drsbru['mean'] = mean_std_drsbru['mean'].round(2)
mean_std_drsbru['std'] = mean_std_drsbru['std'].round(2)
mean_std_drsbru

# MEAN VALUES - COMBINED

In [ ]:
mean_std_both = df_both_acc.groupby(['radiomics', 'model'])['value'].agg(['mean', 'std']).reset_index()
mean_std_both['mean'] = mean_std_both['mean'].round(2)
mean_std_both['std'] = mean_std_both['std'].round(2)
mean_std_both

In [ ]:
df_both_acc

In [ ]:
df_both_pr['tag'] = df_both_pr['tag'].str.replace('acc', '')
df_both_pr['radiomics'] = df_both_pr['tag'].str.contains(r'RADIOMICS|rad').map({True: 'True', False: 'False'})
df_both_pr['precision'] = df_both_pr['tag'].str.contains(r'precision').map({True: 'True', False: 'False'})
df_both_pr['model'] = df_both_pr['tag'].str.replace(r'RADIOMICS|rad|precision|recall', '', regex=True)
df_both_pr['model'] = df_both_pr['model'].str.replace(r'(B3D CNN \+|3D CNN \+)', '', regex=True)
df_both_pr['model'] = df_both_pr['model'].str.strip()
df_both_pr

In [ ]:
models_include = ["Baseline", "Ensemble", "NN", "SVM"]
df_both_filter = df_both_pr[
    (df_both_pr['radiomics'] == 'False') &
    (df_both_pr['model'].isin(models_include))
    ]

In [ ]:
df_both_filter['value'] = (
    df_both_filter['value']
    .str.replace(r'[\[\]]', '', regex=True)    # Remove square brackets
    .str.replace(r'\s{2,}', ' ', regex=True)   # Replace multiple spaces with single space
    .str.strip()                               # Remove leading/trailing spaces
)

df_both_filter[['1', '2', '3', '4', '5']] = df_both_filter['value'].str.split(" ", expand=True).astype(float)


In [ ]:
df_grouped = (
    df_both_filter
    .groupby(['radiomics', 'precision', 'model'])[['1', '2', '3', '4', '5']]
    .mean()
    .round(2)
    .reset_index()
)
df_grouped


In [ ]:
confusion_matrix_ensemble = [
    [28, 21, 2, 0, 0],
    [8, 54, 12, 0, 0],
    [2, 21, 22, 1, 0],
    [0, 1, 4, 11, 2],
    [0, 0, 0, 1, 10]
]

confusion_matrix_ensemble_df = pd.DataFrame(confusion_matrix_ensemble, range(1, 6), range(1,6))

sns.heatmap(confusion_matrix_ensemble_df, annot=True, annot_kws={"size": 12}, cmap='Blues', cbar=False) # font size
plt.title("Ensemble Model Validation Predictions Across 10 Folds")
plt.ylabel("True")
plt.xlabel("Predicted")

plt.savefig('ensemble_confusion_matrix.pdf', bbox_inches='tight')
plt.show()



In [ ]:
confusion_matrix_nn = [
    [21,  29, 1, 0, 0],
    [11, 54, 8, 0, 1],
    [3, 24, 15, 4, 0],
    [1,1,6,5,5],
    [0,0,0,5,6]
]

confusion_matrix_nn_df = pd.DataFrame(confusion_matrix_nn, range(1, 6), range(1,6))

sns.heatmap(confusion_matrix_nn_df, annot=True, annot_kws={"size": 12}, cmap='Blues', cbar=False) # font size
plt.title("Neural Network Validation Predictions Across 10 Folds")
plt.ylabel("True")
plt.xlabel("Predicted")

plt.savefig('nn_confusion_matrix.pdf', bbox_inches='tight')
plt.show()

In [ ]:
confusion_matrix_2d_resnet = [
    [17,34,0,0,0],
    [13,53,8,0,0],
    [3,19,17,3,4],
    [0,0,6,6,6],
    [1,0,2,4,4],
]

confusion_matrix_2dresnet_df = pd.DataFrame(confusion_matrix_2d_resnet, range(1, 6), range(1,6))

sns.heatmap(confusion_matrix_2dresnet_df, annot=True, annot_kws={"size": 12}, cmap='Blues', cbar=False) # font size
plt.title("2D ResNet18 Validation Predictions Across 10 Folds")
plt.ylabel("True")
plt.xlabel("Predicted")

plt.savefig('2dresnet_confusion_matrix.pdf', bbox_inches='tight')
plt.show()

In [ ]:
confusion_matrix_svm = [
    [26,23,2,0,0],
    [7,59,7,1,0],
    [1,19,25,1,0],
    [0,1,5,11,1],
    [0,0,0,2,9]
]


confusion_matrix_svm_df = pd.DataFrame(confusion_matrix_svm, range(1, 6), range(1,6))

sns.heatmap(confusion_matrix_svm_df, annot=True, annot_kws={"size": 12}, cmap='Blues', cbar=False) # font size
plt.title("SVM Validation Predictions Across 10 Folds")
plt.ylabel("True")
plt.xlabel("Predicted")

plt.savefig('svm_confusion_matrix.pdf', bbox_inches='tight')
plt.show()